In [1]:
%pylab inline

import pandas as pd

from numpy import isnan

from nltk.corpus import stopwords

from textblob import Word
from textblob import TextBlob
from textblob import Blobber
from textblob.tokenizers import WordTokenizer

Populating the interactive namespace from numpy and matplotlib


In [2]:
data=pd.read_csv("Assignment 3 Edmunds Posts.csv", usecols=[0])

###Standardize multiple different ways of referring to the same car.

In [3]:
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusES","ES"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("ES330","ES"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LS460","LS"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LS470","LS"))

data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusLS","LS"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusRX","RX"))

###Define necessary functions and lists. 

In [4]:
sw=stopwords.words('english')

sw.remove('not')

models_skinny=["es","ls","rx","a8","a6","3series",\
               "5series","7series","xj","sclass"] 

In [5]:
#tokenize words, remove stop words, lemmatize remaining words, and return them in lowercase
def tKnzr (xstring):
    global sw
    tokens=list(TextBlob(str(xstring)).words)
    removeStopWords=[word for word in tokens if word.lower() not in sw]
    lemmaed=[Word(w).lemmatize() for w in removeStopWords]
    lowercase=[word.lower() for word in lemmaed]
    return lowercase

In [6]:
#find words within a certain distance of models
def finder (tokensList, modelList, numberWords):
    blanklist=[]
    for i in xrange(len(tokensList)):
        if tokensList[i] in modelList:
            blanklist.append(tokensList[i-numberWords:i+1+numberWords])
    return blanklist

In [7]:
#compile sentiments into a single analyzable dataframe
def sentiment_compiler(postseries,modelsList,numberofwords):
    tb=Blobber(tokenizer=WordTokenizer())
    
    newseries=postseries.apply(tKnzr)
    newseries=newseries.apply(lambda x: finder(x,modelsList, numberofwords))
    newseries=newseries.apply(lambda x: [' '.join(inner) for inner in x])
    
    models=[" "+x+" " for x in modelsList]
    df=pd.DataFrame(columns=modelsList).join(newseries, how="outer")
    
    for index,l in enumerate(newseries):
        for string in l: #looping over all the strings in model_strings
                for model, model_skinny in zip(models,modelsList): #loop over all the models in
model list
                    if model in string: #check if model is in a particular list
                        if isnan(df[model_skinny].iloc[index]): #correcting for neutral
                            df[model_skinny].iloc[index]=0
                        df[model_skinny].iloc[index]+=tb(string).sentiment[0]  
    return df

###Run functions on dataset

In [8]:
q=sentiment_compiler(data['Posts'],models_skinny,2)

In [9]:
q.head(1) #this review compares the s-class to the a8.

,es,ls,rx,a8,a6,3series,5series,7series,xj,sclass,Posts
0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-0.166667,"[previous '01 sclass pre-'03 refresh, one issu..."
